# Spacy/Medspacy IAA

## Resources

Prodigy forum answer about IAA for spans https://support.prodi.gy/t/proper-way-to-calculate-inter-annotator-agreement-for-spans-ner/5760

Spacy scorer object https://spacy.io/api/scorer

## End Goal

### Functionality

Provide a collection of methods to evaluate IAA between _n_ arbitrary spacy `doc` objects. Provide methods that aid in error analysis such as providing lists of differences.

Priorities:
* Pairwise F1
    * configurable strict/loose matching
    * configurable inclusion of labels/attributes (calculate just span vs span+class agreement)

* Imported python files
    * reasonable docstrings on methods/classes
    
* Unit tests
    * add CI to repo for automated testing later

Extra features:
* List of differences between docs
* 

Expected challenges
* Spacy scorer functions are useful, but _only_ do strict span matching
* Fewer resources (obviously?) available for comparisons between 3+ docs


# Testing Section

In [ ]:
import spacy

In [ ]:
#!python -m spacy download en_core_web_md

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp2 = spacy.load("en_core_web_md")

In [ ]:
doc = nlp("this is a test document made in utah or mississippi, or salt lake city.")

In [ ]:
doc.ents

In [ ]:
doc2 = nlp2("this is a test document made in utah or mississippi, or salt lake city.")

In [ ]:
doc2.ents

In [ ]:
from spacy.tokens import Span
spand = list()
spand += [Span(doc, 2, 4, label="PERSON"),Span(doc,7,8,label="GPE"),Span(doc,9,10,label="PERSON"),Span(doc,13,14,label="GPE"),Span(doc,14,15,label="GPE")]

# Add the span to the doc's entities
doc.ents = spand

# Print entities' text and labels
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
#Run below cells before this
tp,fp,fn = agreement(doc,doc2,1,1)
print(tp,fp,fn)
print(pairwise_f1(tp,fp,fn))

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

test_str = "This is a test document. For testing lol."
test_str_2 = "This is a test document. For testing lol."

doc = nlp(test_str)
doc2 = nlp(test_str)

from spacy.tokens import SpanGroup

spans = [doc[0:1], doc[1:3]]
group = SpanGroup(doc, name="errors", spans=spans, attrs={"annotator": "matt"})
doc.spans["errors"] = group
group = SpanGroup(doc, name="entity1", spans=spans, attrs={"annotator": "John"})
#doc.spans["entity"] = group


spans = [doc2[0:1], doc2[1:5],doc2[4:5]]
group = SpanGroup(doc2, name="errors", spans=spans, attrs={"annotator": "matt"})
doc2.spans["errors"] = group
group = SpanGroup(doc2, name="entity1", spans=spans, attrs={"annotator": "John"})
#doc2.spans["entity"] = group

In [ ]:
from spacy.tokens import Span
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp("this is a test document made in utah or mississippi, or salt lake city.")

spand = list()
spand += [Span(doc, 2, 4, label="PERSON"),Span(doc,7,8,label="GPE"),Span(doc,9,10,label="PERSON"),Span(doc,13,14,label="GPE"),Span(doc,14,15,label="GPE")]

# Add the span to the doc's entities
doc.ents = spand

print([(ent.text, ent.label_) for ent in doc.ents])

from spacy.tokens import SpanGroup

spans = [doc[7:8], doc[8:10]]
group = SpanGroup(doc, name="errors", spans=spans, attrs={"annotator": "matt"})
doc.spans["errors"] = group
print(doc.spans['errors'])
print(doc.spans['errors'][0].label_)

In [ ]:
print(doc.spans['errors'])
for span in doc.spans['errors']:
    print('what')

In [ ]:
agreement(doc.ents,doc.spans['errors'],labels=0)

In [ ]:
agreement(doc,doc2,ent_or_span='ent')

In [ ]:
type(doc.ents[0]) is spacy.tokens.span.Span
type(doc.spans['errors']) is spacy.tokens.span_group.SpanGroup

type(doc)

# Code

In [45]:
from quicksectx import IntervalNode, IntervalTree, Interval #note that you need the quicksectx library

#In order to make the code a little more adaptable for situations of multiple overlapping entities, as well as for 
#transparency and testing the code, I wrote the overlaps code to output a mapping of which entities are being matched. 
#Then agreement can parse this output for how many valid overlaps exist.

#This makes the code a little more complicated to understand, but I think it makes everything more transparent and adaptable.

def overlaps(doc1_ents, doc2_ents,labels=1):
    '''Calculates overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()
    
    tree = IntervalTree()
    for index2,ent2 in enumerate(doc2_ents):
        tree.add(ent2.start_char,ent2.end_char,index2)
    
    for index1,ent1 in enumerate(doc1_ents):
        matches = tree.search(ent1.start_char,ent1.end_char)
        for match in matches:
            index2 = match.data #match.data is the index of doc2_ents
            if ((labels == 0) | (doc2_ents[index2].label_ == ent1.label_)):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches

In [46]:
### This is the old, less efficient code. The newer code uses a tree search instead of the nested for-loop.

def old_overlaps(doc1_ents, doc2_ents,labels):
    '''Old code for calculating overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()

    for index1,ent1 in enumerate(doc1_ents):
        for index2,ent2 in enumerate(doc2_ents):
            if (ent1.end_char >= ent2.start_char) & (ent1.start_char <= ent2.end_char) & ((labels==0) | (ent1.label_ == ent2.label_)):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches
    

In [47]:
def exact_match(doc1_ents, doc2_ents, labels):
    '''calculate whether two ents have exact overlap
    returns bool
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()

    doc1_ent_dict = dict()
    doc2_ent_dict = dict()
    
    for index1,ent1 in enumerate(doc1_ents):
        if labels == 1: #If checking for labels, then include this in the tuple's to-be-compared elements
            doc1_ent_dict[(ent1.start_char,ent1.end_char,ent1.label_)] = index1
        else:
            doc1_ent_dict[(ent1.start_char,ent1.end_char)] = index1
            
    for index2,ent2 in enumerate(doc2_ents):
        if labels == 1:    
            doc2_ent_dict[(ent2.start_char,ent2.end_char,ent2.label_)] = index2
        else:
            doc2_ent_dict[(ent2.start_char,ent2.end_char)] = index2
        
    doc1_ent_set = set(doc1_ent_dict.keys())
    doc2_ent_set = set(doc2_ent_dict.keys())
    
    matched_ents = doc1_ent_set.intersection(doc2_ent_set)
    
    for match in matched_ents:
        index1 = doc1_ent_dict[match]
        index2 = doc2_ent_dict[match]
        doc1_matches[index1] = [index2]
        doc2_matches[index2] = [index1]
        
    return doc1_matches, doc2_matches
    

In [48]:
def agreement(doc1, doc2, loose=1, labels=1, ent_or_span = 'ent'):
    '''Calculates confusion matrix for agreement between two documents.
    
       returns true positive, false positive, and false negative
    '''
    if (type(doc1) is tuple) or (type(doc1) is spacy.tokens.span_group.SpanGroup) and \
    (type(doc2) is tuple) or (type(doc2) is spacy.tokens.span_group.SpanGroup):
        doc1_ents = doc1
        doc2_ents = doc2
    elif (type(doc1) is spacy.tokens.doc.Doc) and (type(doc2) is spacy.tokens.doc.Doc):
        if ent_or_span == 'ent':
            doc1_ents = doc1.ents
            doc2_ents = doc2.ents
        elif ent_or_span == 'span':
            if len(doc1.spans) > 1:
                #raise error
                print("Error: cannot distinquish which span group to use from doc1.")
                return
            else:
                span_group = list(doc1.spans.keys())[0]
                doc1_ents = doc1.spans[span_group]
                doc2_ents = doc2.spans[span_group]
        else:
            #raise error
            print("Error: Must select 'span' or 'ent' for ent_or_span option.")
            return
    else:
        #raise error
        print("Error: Input must be of type 'tuples', 'spacy.tokens.span_group.SpanGroup', or 'spacy.tokens.doc.Doc'")
        return
        
    if loose:
        doc1_matches, doc2_matches = overlaps(doc1_ents, doc2_ents, labels)
    else:
        doc1_matches, doc2_matches = exact_match(doc1_ents, doc2_ents, labels)
    
    return conf_matrix(doc1_matches,doc2_matches,len(doc1_ents),len(doc2_ents))


In [49]:
def conf_matrix(doc1_matches,doc2_matches,doc1_ent_num,doc2_ent_num):

    doc1_match_num = len(doc1_matches.keys())
    doc2_match_num = len(doc2_matches.keys())
    
    duplicate_matches = 0
    for value in doc2_matches.values():
        duplicate_matches += len(value) - 1
    
    tp = doc1_match_num - duplicate_matches #How many entity indices from doc1 matched, minus duplicated matches
    fp = doc2_ent_num - doc2_match_num #How many entities from doc2 that didn't match
    fn = doc1_ent_num - doc1_match_num #How many entities from doc1 that didn't match
    
    return (tp,fp,fn)

In [50]:
def pairwise_f1(tp,fp,fn):
    '''calculate f1 given true positive, false positive, and false negative values'''
    
    return (2*tp)/float(2*tp+fp+fn)

In [58]:
def corpus_agreement(docs1, docs2, loose=1, labels=1,ent_or_span='ent'):
    '''calculate f1 over an entire corpus of documents'''
    corpus_tp, corpus_fp, corpus_fn = (0,0,0)
    
    if type(docs1[0]) is spacy.tokens.doc.Doc:
        for i, doc1 in enumerate(docs1):
            tp,fp,fn = agreement(doc1, docs2[i],loose,labels,ent_or_span)
            corpus_tp += tp
            corpus_fp += fp
            corpus_fn += fn
    elif type(docs1) is pandas.core.frame.DataFrame:
        for doc_name in docs1['doc name'].unique():
            docs1_df = docs1[docs1['doc name'] == doc_name]
            docs2_df = docs2[docs2['doc name'] == doc_name]
            doc1_matches,doc2_matches = df_overlaps(docs1_df,docs2_df)
            tp,fp,fn = conf_matrix(doc1_matches,doc2_matches,docs1_df.shape[0],docs2_df.shape[0])
            corpus_tp += tp
            corpus_fp += fp
            corpus_fn += fn
    else:
        #raise error
        print('Input Error: Input must be iterable of spacy documents, or dataframe.')
        return
    
    data = {'IAA' : [pairwise_f1(corpus_tp,corpus_fp,corpus_fn)], 'Recall' : [tp/float(tp+fp)], 'Precision' : [tp/float(tp+fn)],\
           'True Positives' : [tp] , 'False Positives' : [fp], 'False Negative' : [fn]}
    
    return pd.DataFrame(data)

# Tutorial

In this tutorial I will go over the basic, front-end usage of calculating IAA between 2 annotators.

In [70]:
import spacy
import medspacy
nlp1 = spacy.load("en_core_web_sm")
nlp2 = spacy.load("en_core_web_md")
#!python -m spacy download en_core_web_sm
#!python -m spacy download en_core_web_md

#Note for John: Get better examples or make my own entities
doc1 = nlp1("this is a test document made in utah or mississippi, or salt lake city.")
doc2 = nlp2("this is a test document made in utah or mississippi, or salt lake city.")

print('doc1.ents: ',doc1.ents)
print('doc2.ents: ',doc2.ents)

doc1.ents:  (utah, mississippi)
doc2.ents:  (utah, mississippi, lake city)


Above we made two documents using spacy's NER packages. Document 2 added more entities than document 1. Let's calculate the IAA between these documents!

In [71]:
corpus_agreement([doc1],[doc2])

,IAA,Recall,Precision,True Positives,False Positives,False Negative
0,0.8,0.666667,1.0,2,1,0


'corpus_agreement' calculates the agreement between two lists of documents. Note the brackets around 'doc1' and 'doc2' so they are passed in as lists of size 1 each.

'corpus_agreement' can also take options to be more flexible with other IAA methods. Below are the arguments:

### corpus_agreement(docs1, docs2, loose=1, labels=1,ent_or_span='ent')

docs1: list of spacy documents

docs2: list of spacy documents with same order as docs1

loose: Boolean for allowing loose matching. '1' indicates that any overlap between two spans/entities is counted towards IAA. '0' indicates that only exact matches will be allowed.

labels: Boolean to include labels when matching. Uses the .label_ attribute of entities/spans to access labels.

ent_or_span: string of whether spans or entities are being compared. If set to 'ent', code will iterate through doc1.ents and doc2.ents. If set to 'span', code will iterate through the spans within doc1 and doc2's first span group. 'span' only works if doc1 has one span group. This option may be extended to include doc._ .concepts option, or to allow multiple span groups to be compared.

Internally, corpus_agreement is calling the 'agreement' and 'pairwise_f1' functions on each pair of document in the lists. We can instead choose to call these functions separate

# Code in Development

In [1]:
import pandas as pd
john_df = pd.read_pickle('./df_John.pkl')
#john_df = john_df[john_df['Concept Label'] == 'Symptom']
mengke_df = pd.read_pickle('./df_Mengke.pkl')

In [8]:
from quicksectx import IntervalNode, IntervalTree, Interval #note that you need the quicksectx library

#In order to make the code a little more adaptable for situations of multiple overlapping entities, as well as for 
#transparency and testing the code, I wrote the overlaps code to output a mapping of which entities are being matched. 
#Then agreement can parse this output for how many valid overlaps exist.

#This makes the code a little more complicated to understand, but I think it makes everything more transparent and adaptable.

def df_overlaps(docs1_df, docs2_df,labels=1):
    '''Calculates overlapping entities between two spacy documents. Also checks for matching labels if label=1.
    
    Return:
        Dictionaries with the mapping of matching entity indices:
            keys: entity index from one annotation
            value: matched entity index from other annotation
        
        Ex: "{1 : [2] , 3 : [4,5]}" means that entity 1 from doc1 matches entity 1 in doc2, and entity 3 in doc1 matches 
        entity 4 and 5 from doc2.
    '''
    
    doc1_matches = dict()
    doc2_matches = dict()
    
    tree = IntervalTree()
    for index2,row2 in docs2_df.iterrows():
        tree.add(row2['start loc'],row2['end loc'],index2)
    
    for index1,row1 in docs1_df.iterrows():
        matches = tree.search(row1['start loc'],row1['end loc'])
        for match in matches:
            index2 = match.data #match.data is the index of doc2_ents
            if ((labels == 0) | (docs2_df.loc[index2,'Concept Label'] == row1['Concept Label'])):
                if index1 not in doc1_matches.keys():
                    doc1_matches[index1] = [index2]
                else:
                    doc1_matches[index1].append(index2)
                if index2 not in doc2_matches.keys():
                    doc2_matches[index2] = [index1]
                else:
                    doc2_matches[index2].append(index1)
                
    return doc1_matches, doc2_matches

In [38]:
def df_corpus_agreement(docs1, docs2, loose=1, labels=1,ent_or_span='ent'):
    '''calculate f1 over an entire corpus of documents'''
    corpus_tp, corpus_fp, corpus_fn = (0,0,0)
    
    for doc_name in docs1['doc name'].unique():
        docs1_df = docs1[docs1['doc name'] == doc_name]
        docs2_df = docs2[docs2['doc name'] == doc_name]
        doc1_matches,doc2_matches = df_overlaps(docs1_df,docs2_df)
        tp,fp,fn = conf_matrix(doc1_matches,doc2_matches,docs1_df.shape[0],docs2_df.shape[0])
        corpus_tp += tp
        corpus_fp += fp
        corpus_fn += fn
    
    print('corpus tp: ',corpus_tp,'\ncorpus fp: ',corpus_fp,'\ncorpus fn: ',corpus_fn)
    
    print(tp)
    print(corpus_tp)
    
    #print("Not in doc2 annotations:\n")
    for index,row in docs1.iterrows():
        if (index not in doc2_matches.keys()):
            #print(row['Span Text'])
            break
    
    return pairwise_f1(corpus_tp,corpus_fp,corpus_fn)

In [39]:
df_corpus_agreement(john_df,mengke_df,1)

corpus tp:  376 
corpus fp:  176 
corpus fn:  842
8
376


0.4248587570621469

In [40]:
john_df_symptoms = john_df[john_df['Concept Label'] == 'Symptom']
mengke_df_symptoms = mengke_df[mengke_df['Concept Label'] == 'Symptom']

df_corpus_agreement(john_df_symptoms,mengke_df_symptoms)

corpus tp:  348 
corpus fp:  157 
corpus fn:  563
8
348


0.4915254237288136

In [55]:
corpus_agreement(john_df_symptoms,mengke_df_symptoms,1)

KeyError: 0

In [52]:
type(john_df)

pandas.core.frame.DataFrame